In [1]:
import zfit

/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(


In [2]:
file_bkg_ele_B0pip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/B0eepip4Streams.root"
file_signal_ele_B0pip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/B0eepip200000.root"


file_bkg_mu_B0pip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/B0mumupip4Streams.root"
file_signal_mu_B0pip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/B0mumupip200000.root"


In [3]:
file_bkg_ele_Bppip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/Bpeepip4Streams.root"
file_signal_ele_Bppip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/Bpeepip200000.root"


file_bkg_mu_Bppip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/Bpmumupip4Streams.root"
file_signal_mu_Bppip="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/Bpmumupip200000.root"


In [4]:
file_bkg_ele_Bppi0="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/Bpeepi04Streams.root"
file_signal_ele_Bppi0="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/Bpeepi0200000.root"


file_bkg_mu_Bppi0="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/background/Bpmumupi04Streams.root"
file_signal_mu_Bppi0="/pnfs/desy.de/belle/local/user/msobotzi/p5prime/FilesAfterAcceptance/MC15/signal/Bpmumupi0200000.root"


In [5]:
class trimVars():
    @staticmethod
    def trimCosTheta_L(df, Btree, pi, ll, q2):
        if int(Btree) == 511:
            if q2 == 1:
                df = df.query('abs(cosTheta_L) < 0.75')
            if q2 == 4:
                df = df.query('abs(cosTheta_L) < 0.95')
        
        if int(Btree) == 521:
            if pi == 'pip':
                if q2 == 1:
                    df = df.query('abs(cosTheta_L) < 0.75')
                elif q2 == 4:
                    df = df.query('abs(cosTheta_L) < 0.95')
            if pi =='pi0':
                if q2 == 1:
                    df = df.query('abs(cosTheta_L) < 0.75')
                elif q2 != 4:
                    df = df.query('abs(cosTheta_L) < 0.9')
        return df
    
    @staticmethod
    def trimCosTheta_K(df, Btree, pi, ll, q2):
        if int(Btree) == 511:
            if q2 == 1:
                df = df.query('cosTheta_K > -0.9')
            elif q2 != 4:
                df = df.query('cosTheta_K > -0.9')
            else:
                df = df.query('abs(cosTheta_K) < 0.95')
                
        if int(Btree) == 521:
            if q2 == 1:
                if pi == 'pip':
                    df = df.query('cosTheta_K > -0.9')
                else:
                    df = df.query('cosTheta_K > 0.9')
            elif q2 != 4:
                if pi == 'pip':
                    df = df.query('cosTheta_K > -0.9')
                elif pi == 'pi0':
                    df = df.query('cosTheta_K > -0.9')
                    
            elif q2 == 4:
                if pi == 'pip':
                    df = df.query('abs(cosTheta_K) > -0.95')
                if pi == 'pi0':
                    df = df.query('abs(cosTheta_K) < 0.9')
        return df
    
    @staticmethod
    def trimmers(df, Btree, pi, ll, q2):
        df = trimVars.trimCosTheta_L(df, Btree, pi, ll, q2)
        df = trimVars.trimCosTheta_K(df, Btree, pi, ll, q2)
        return df

def process_MC_data_combined(file_path_signal, file_path_background, particle_type, Btree, pi, ll, q2, tree_name, weight_column,sample_size=None):
    
    def process_mc_data(file_path, q2):
        columns = ['q2', 'Mbc', 'deltaE','phi_B', 'cosTheta_L', 'cosTheta_K', 'NNPass', 'bestCandidate','isSignal', weight_column, 'q2reg', 'lm_genMotherPDG', 'lp_genMotherPDG', 'Kst_genMotherPDG', 'Kst_isSignal', 'lm_mcPDG', 'lp_mcPDG','lm_genMotherPDG_1']
        with uproot.open(f"{file_path}:{tree_name}") as file:
            df = file.arrays(columns, library='pd')
        mc_data = trimVars.trimmers(df, Btree, pi, ll, q2)
        mc_cut = mc_data.query('NNPass==1')
        mc_cut_best_candidate = mc_cut.query('bestCandidate == 1')
        return mc_cut_best_candidate
        
    mc_cut_best_candidate_signal = process_mc_data(file_path_signal, q2)
    mc_cut_best_candidate_background = process_mc_data(file_path_background, q2)

    if q2 == 111 or q2 == 112:
        
        mc_cut_best_candidate_combine = pd.concat([mc_cut_best_candidate_signal, mc_cut_best_candidate_background])

        mc_q2cut_background = mc_cut_best_candidate_combine.query(f"q2reg == {q2}")

        if particle_type == 'ele':
            cut = f"lm_genMotherPDG == {443 if q2 == 11 else 100443} and lp_genMotherPDG == {443 if q2 == 11 else 100443} and abs(Kst_genMotherPDG) == {Btree} and Kst_isSignal == 1 and abs(lm_mcPDG) == 11 and abs(lp_mcPDG) == 11 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        elif particle_type == 'mu':
            cut = f"lm_genMotherPDG == {443 if q2 == 11 else 100443} and lp_genMotherPDG == {443 if q2 == 11 else 100443} and abs(Kst_genMotherPDG) == {Btree} and Kst_isSignal == 1 and abs(lm_mcPDG) == 13 and abs(lp_mcPDG) == 13 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        else:
            raise ValueError(f"Unknown particle_type: {particle_type}")

        deltaE_range = (-0.15, 0.15)

        mc_signal_q2cut_jpsi_kst_wot_filter = mc_q2cut_background.query(cut)
        mc_signal_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst_wot_filter.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')

        mc_bkg_q2cut_jpsi_kst_wot_filter = mc_q2cut_background.query(f'not ({cut})')
        mc_bkg_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst_wot_filter.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')

        mc_signal_weight_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst[weight_column]
        mc_bkg_weight_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst[weight_column]

        # Data
        mc_signal_data = mc_signal_q2cut_jpsi_kst
        mc_bkg_data = mc_bkg_q2cut_jpsi_kst

        # Weight Data
        mc_signal_weight = mc_signal_weight_q2cut_jpsi_kst
        mc_bkg_weight = mc_bkg_weight_q2cut_jpsi_kst

        return mc_signal_data, mc_bkg_data, mc_signal_weight, mc_bkg_weight
        
    else:
        #mc_cut_best_candidate_combine = pd.concat([mc_cut_best_candidate_signal, mc_cut_best_candidate_background])

        if q2 in [1, 2, 3, 4,11, 12]:
            mc_q2cut_signal = mc_cut_best_candidate_signal.query(f"q2reg == {q2}")
            mc_q2cut_background = mc_cut_best_candidate_background.query(f"q2reg == {q2}")
        elif q2 == 0:
            included_q2reg_values = [1, 2, 3, 4]
            excluded_q2reg_values = [11, 12]
            
            mc_q2cut_signal = mc_cut_best_candidate_signal.query(f'q2reg in {included_q2reg_values} and q2reg not in {excluded_q2reg_values}')
            mc_q2cut_background = mc_cut_best_candidate_background.query(f'q2reg in {included_q2reg_values} and q2reg not in {excluded_q2reg_values}')

        elif q2 == 10:
            included_q2reg_values = [1, 2, 3, 4, 11, 12]
            
            mc_q2cut_signal = mc_cut_best_candidate_signal.query(f'q2reg in {included_q2reg_values}')
            mc_q2cut_background = mc_cut_best_candidate_background.query(f'q2reg in {included_q2reg_values}')
           
        else:
            raise ValueError("Invalid q2 value")

        deltaE_range = (-0.15, 0.15)

        mc_particle_type_signal = mc_q2cut_signal.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')
        
        mc_particle_type_only_signal = mc_particle_type_signal.query('isSignal==1')
        
        if sample_size is not None:
            mc_particle_type_only_signal = mc_particle_type_only_signal.sample(n=sample_size, random_state=i) # Adjust random_state as needed
        
        mc_particle_type_scf = mc_particle_type_signal.query('isSignal!=1')

        mc_particle_type_bkg = mc_q2cut_background.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')

        mc_particle_type_signal_weight = mc_particle_type_signal[weight_column]
        mc_particle_type_only_signal_weight = mc_particle_type_only_signal[weight_column]
        mc_particle_type_scf_weight = mc_particle_type_scf[weight_column]

        mc_particle_type_bkg_weight = mc_particle_type_bkg[weight_column]
        
        return (mc_particle_type_signal, mc_particle_type_only_signal, mc_particle_type_scf, mc_particle_type_bkg,
                mc_particle_type_signal_weight, mc_particle_type_only_signal_weight, mc_particle_type_scf_weight,
                mc_particle_type_bkg_weight)

def process_MC_data_combined_jpsi(file_path_jpsi, tree_name, weight_column):
    def process_mc_data(file_path):
        columns = ['q2', 'Mbc', 'deltaE', 'NNPass', 'bestCandidate', weight_column, 'isSignal']
        with uproot.open(f"{file_path}:{tree_name}") as file:
            df = file.arrays(columns, library='pd')
        mc_cut = df.query('NNPass==1')
        mc_cut_best_candidate = mc_cut.query('bestCandidate == 1')
        return mc_cut_best_candidate

    mc_cut_best_candidate_background = process_mc_data(file_path_jpsi)

    # Apply range cut for deltaE
    deltaE_range = (-0.15, 0.15)

    # Signal
    mc_signal_q2cut_jpsi_kst_wot_filter = mc_cut_best_candidate_background
    mc_signal_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst_wot_filter[(mc_signal_q2cut_jpsi_kst_wot_filter['deltaE'] >= deltaE_range[0]) & (mc_signal_q2cut_jpsi_kst_wot_filter['deltaE'] <= deltaE_range[1])]
    mc_only_signal_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst.query('isSignal==1')
    mc_scf_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst.query('isSignal!=1')

    # Weight
    mc_signal_weight_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst[weight_column]
    mc_only_signal_weight_q2cut_jpsi_kst = mc_only_signal_q2cut_jpsi_kst[weight_column]
    mc_scf_weight_q2cut_jpsi_kst = mc_scf_q2cut_jpsi_kst[weight_column]

    # Data
    mc_signal_data = mc_signal_q2cut_jpsi_kst
    mc_only_signal_data = mc_only_signal_q2cut_jpsi_kst
    mc_scf_data = mc_scf_q2cut_jpsi_kst

    # Weight Data
    mc_signal_weight = mc_signal_weight_q2cut_jpsi_kst
    mc_only_signal_weight = mc_only_signal_weight_q2cut_jpsi_kst
    mc_scf_weight = mc_scf_weight_q2cut_jpsi_kst

    return mc_signal_data, mc_only_signal_data, mc_scf_data, mc_signal_weight, mc_only_signal_weight, mc_scf_weight


In [6]:
def angular_gen_data_JPsi(file_path_background, Btree, pi, ll, q2,tree_name):
    def process_mc_data(file_path, Btree, pi, ll, q2):
        columns = ['q2', 'Mbc', 'deltaE','phi_B', 'cosTheta_L', 'cosTheta_K', 'lm_genMotherPDG', 'lp_genMotherPDG', 'Kst_genMotherPDG', 'Kst_isSignal', 'lm_mcPDG', 'lp_mcPDG','lm_genMotherPDG_1']
        with uproot.open(f"{file_path}:{tree_name}") as file:
            df = file.arrays(columns, library='pd')
        mc_data_with_trim = trimVars.trimmers(df, Btree, pi, ll, q2)
        mc_data_without_trim = df
        
        return mc_data_with_trim, mc_data_without_trim

    mc_data_with_trim, mc_data_without_trim = process_mc_data(file_path_background, Btree, pi, ll, q2)
    
    def apply_cuts(mc_data, ll):
        if ll == 'ee':
            cut = f"lm_genMotherPDG == 443 and lp_genMotherPDG == 443 and abs(Kst_genMotherPDG) == {Btree} and Kst_isSignal == 1 and abs(lm_mcPDG) == 11 and abs(lp_mcPDG) == 11 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        elif ll == 'mumu':
            cut = f"lm_genMotherPDG == 443 and lp_genMotherPDG == 443 and abs(Kst_genMotherPDG) == {Btree} and Kst_isSignal == 1 and abs(lm_mcPDG) == 13 and abs(lp_mcPDG) == 13 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        else:
            raise ValueError(f"Unknown ll: {ll}")
        
        deltaE_range = (-0.15, 0.15)
        
        mc_signal_q2cut_jpsi_kst_wot_filter = mc_data.query(cut)
        mc_signal_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst_wot_filter[(mc_signal_q2cut_jpsi_kst_wot_filter['deltaE'] >= deltaE_range[0]) & (mc_signal_q2cut_jpsi_kst_wot_filter['deltaE'] <= deltaE_range[1])]
        
        return mc_signal_q2cut_jpsi_kst
    
    mc_signal_data_with_trim = apply_cuts(mc_data_with_trim, ll)
    mc_signal_data_without_trim = apply_cuts(mc_data_without_trim, ll)
    
    return mc_signal_data_with_trim, mc_signal_data_without_trim

In [7]:
def plot_histogram(ax, data_dict, labels, data_weight, xlabel, ylabel, title):
    for label in labels:
        ax.hist(data_dict[label], weights=data_weight[label], bins=100, alpha=0.5, label=label)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.set_title(title)

In [8]:
class p5_prime_pdf(zfit.pdf.ZPDF):
    _N_OBS = 3  # dimension, can be omitted
    _PARAMS = ['param_fl','param_s3','param_p5']  # the name of the parameters

    def _unnormalized_pdf(self, x):
        x, y, z = x.unstack_x()  # x=costk, y=costl, z=phib
        param_fl = self.params['param_fl']
        param_s3 = self.params['param_s3']
        param_p5 = self.params['param_p5']
        
        return (9/(8*np.pi))*((3/4)*(1-param_fl)* (1-x**2) + param_fl * x**2 + \
               (1/4) *(1 - param_fl)* (1-x**2)* (2 * y**2 - 1)- \
               param_fl * x**2 * (2 * y**2 - 1) + param_s3 *(1-x**2)*(1-y**2) * tf.cos(2*z)+ \
               tf.sqrt(param_fl*(1-param_fl)) * param_p5 * 2*tf.sqrt(1-x**2) *x * tf.sqrt(1-y**2)* tf.cos(z))

    
def JPsi_fit_and_integrate(mc_signal_data_with_trim,mc_signal_data_without_trim, param_fl_start=0.5, param_s3_start=0.01, param_p5_start=0.02, floating=True):
    def fold_data(data):
        data['cosTheta_L'] = data['cosTheta_L'].abs()
        phi_b_gr = data.query(f"phi_B > {np.pi}")
        phi_b_gr['phi_B'] = 2 * np.pi - phi_b_gr['phi_B']
        phi_b_kl = data.query(f"phi_B <= {np.pi}")
        folded_data = pd.concat([phi_b_kl, phi_b_gr])
        return folded_data
 
    def p5p_3D_fit(cost_k_data, cost_l_data, phi_b_data, param_fl_start, param_s3_start, param_p5_start,floating=True):
        # create space
        obs_cost_k = zfit.Space('cosTheta_K', limits=(min(cost_k_data), max(cost_k_data)))
        obs_cost_l = zfit.Space('cosTheta_L', limits=(min(cost_l_data), max(cost_l_data)))
        obs_phi_b = zfit.Space('phi_B', limits=(0, np.pi))
        obs = obs_cost_k * obs_cost_l * obs_phi_b

        # data
        df_sig = pd.concat([cost_k_data, cost_l_data, phi_b_data], axis=1)
        df_sig.columns = ['cosTheta_K', 'cosTheta_L', 'phi_B']  # Renaming columns to match observables
        data_fit_sig = zfit.Data.from_pandas(df_sig, obs=obs)

        # parameter variable
        r = np.round(np.random.rand(), decimals=8)

        # parameters
        param_fl = zfit.Parameter(f'param_fl_{r}', param_fl_start, floating=floating)
        param_s3 = zfit.Parameter(f'param_s3_{r}', param_s3_start, floating=floating)
        param_p5 = zfit.Parameter(f'param_p5_{r}', param_p5_start, floating=floating)

        extended = zfit.Parameter(f"extended_{r}", 10000)

        pdf_3D = p5_prime_pdf(param_fl=param_fl, param_s3=param_s3, param_p5=param_p5, obs=obs, extended=extended)
        pdf_3D.update_integration_options(max_draws=10000, tol=1e-4)  # Adjust parameters as needed

        # create NLL
        nll = zfit.loss.ExtendedUnbinnedNLL(model=pdf_3D, data=data_fit_sig)
        minimizer = zfit.minimize.Minuit()
        result = minimizer.minimize(nll)

        if floating == True:
            result.hesse(name='hesse')

        print(result.params)
        print("Fit result valid:")
        print(result.valid)

        return pdf_3D
    
    #with trim
    mc_signal_data_with_trim_fold = fold_data(mc_signal_data_with_trim)
    mc_signal_data_with_trim_weight_fold=len(mc_signal_data_with_trim_fold)
    
    #without trim
    mc_signal_data_without_trim_fold = fold_data(mc_signal_data_without_trim)
    mc_signal_data_without_trim_weight_fold=len(mc_signal_data_without_trim_fold)

    # Example usage with adjusted starting parameters
    param_fl_start = 0.5  # Adjust based on your data 0.5
    param_s3_start = 0.01  # Adjust based on your data -0.01
    param_p5_start = 0.02  # Adjust based on your data 0.02

    # Fit with phi_B greater than np.pi

    pdf_fit_signal_with_trim_gr = p5p_3D_fit(mc_signal_data_with_trim_fold["cosTheta_K"], mc_signal_data_with_trim_fold["cosTheta_L"], mc_signal_data_with_trim_fold["phi_B"],
                                   param_fl_start, param_s3_start, param_p5_start,floating=True)
    pdf_fit_signal_without_trim_gr = p5p_3D_fit(mc_signal_data_without_trim_fold["cosTheta_K"], mc_signal_data_without_trim_fold["cosTheta_L"], mc_signal_data_without_trim_fold["phi_B"],
                               param_fl_start, param_s3_start, param_p5_start,floating=True)



    # Now, let's print the total signal events again
    print("Signal Events for with_trim in MC Data =", mc_signal_data_with_trim_weight_fold)
    
    # Now, let's print the total signal events again
    print("Signal Events for without_trim  in MC Data =", mc_signal_data_without_trim_weight_fold)
    
    cosTheta_K_range_with_trim = np.linspace(-0.9, 1.0, 100)  # Range for cosTheta_K with trim
    cosTheta_K_range_without_trim = np.linspace(-1.0, 1.0, 100)  # Range for cosTheta_K without trim
    cosTheta_L_range = np.linspace(0, 1.0, 100)
    phi_B_range = np.linspace(0, np.pi, 100)

    # Create grids for with trim
    cosTheta_K_grid_with_trim, cosTheta_L_grid, phi_B_grid = np.meshgrid(cosTheta_K_range_with_trim, cosTheta_L_range, phi_B_range)
    points_with_trim = np.vstack([cosTheta_K_grid_with_trim.flatten(), cosTheta_L_grid.flatten(), phi_B_grid.flatten()]).T

    # Create grids for without trim
    cosTheta_K_grid_without_trim, _, _ = np.meshgrid(cosTheta_K_range_without_trim, cosTheta_L_range, phi_B_range)
    points_without_trim = np.vstack([cosTheta_K_grid_without_trim.flatten(), cosTheta_L_grid.flatten(), phi_B_grid.flatten()]).T

    # PDF values without trim
    pdf_values_gr_without_trim = pdf_fit_signal_with_trim_gr.pdf(points_without_trim).numpy().reshape(cosTheta_K_grid_without_trim.shape)
    integrated_pdf_total_without_trim = simps(simps(simps(pdf_values_gr_without_trim, phi_B_range, axis=2), cosTheta_L_range, axis=1), cosTheta_K_range_without_trim, axis=0)

    # PDF values with trim
    pdf_values_gr_with_trim = pdf_fit_signal_with_trim_gr.pdf(points_with_trim).numpy().reshape(cosTheta_K_grid_with_trim.shape)
    integrated_pdf_total_with_trim = simps(simps(simps(pdf_values_gr_with_trim, phi_B_range, axis=2), cosTheta_L_range, axis=1),  cosTheta_K_range_with_trim, axis=0)

    # Calculate proportion of data lost
    proportion_data_lost = (integrated_pdf_total_without_trim - integrated_pdf_total_with_trim) / integrated_pdf_total_without_trim

    # Print the integrated PDF
    print(f"Integrated PDF with trim: {integrated_pdf_total_with_trim}")
    print(f"Integrated PDF without trim: {integrated_pdf_total_without_trim}")
    print(f"Proportion of data lost: {proportion_data_lost:.4f}")


In [9]:
class p5_prime_pdf(zfit.pdf.ZPDF):
    _N_OBS = 3  # dimension, can be omitted
    _PARAMS = ['param_fl','param_s3','param_p5']  # the name of the parameters

    def _unnormalized_pdf(self, x):
        x, y, z = x.unstack_x()  # x=costk, y=costl, z=phib
        param_fl = self.params['param_fl']
        param_s3 = self.params['param_s3']
        param_p5 = self.params['param_p5']
        
        return (9/(8*np.pi))*((3/4)*(1-param_fl)* (1-x**2) + param_fl * x**2 + \
               (1/4) *(1 - param_fl)* (1-x**2)* (2 * y**2 - 1)- \
               param_fl * x**2 * (2 * y**2 - 1) + param_s3 *(1-x**2)*(1-y**2) * tf.cos(2*z)+ \
               tf.sqrt(param_fl*(1-param_fl)) * param_p5 * 2*tf.sqrt(1-x**2) *x * tf.sqrt(1-y**2)* tf.cos(z))

    
def fit_and_integrate(mc_signal_data, param_fl_start=0.5, param_s3_start=0.01, param_p5_start=0.02, floating=True):
    def fold_data(data):
        data['cosTheta_L'] = data['cosTheta_L'].abs()
        phi_b_gr = data.query(f"phi_B > {np.pi}")
        phi_b_gr['phi_B'] = 2 * np.pi - phi_b_gr['phi_B']
        phi_b_kl = data.query(f"phi_B <= {np.pi}")
        folded_data = pd.concat([phi_b_kl, phi_b_gr])
        return folded_data
 
    def p5p_3D_fit(cost_k_data, cost_l_data, phi_b_data, param_fl_start, param_s3_start, param_p5_start,weight,floating=True):
        # create space
        obs_cost_k = zfit.Space('cosTheta_K', limits=(min(cost_k_data), max(cost_k_data)))
        obs_cost_l = zfit.Space('cosTheta_L', limits=(min(cost_l_data), max(cost_l_data)))
        obs_phi_b = zfit.Space('phi_B', limits=(0, np.pi))
        obs = obs_cost_k * obs_cost_l * obs_phi_b

        # data
        df_sig = pd.concat([cost_k_data, cost_l_data, phi_b_data], axis=1)
        df_sig.columns = ['cosTheta_K', 'cosTheta_L', 'phi_B']  # Renaming columns to match observables
        data_fit_sig = zfit.Data.from_pandas(df_sig, obs=obs, weights=weight)

        # parameter variable
        r = np.round(np.random.rand(), decimals=8)

        # parameters
        param_fl = zfit.Parameter(f'param_fl_{r}', param_fl_start, floating=floating)
        param_s3 = zfit.Parameter(f'param_s3_{r}', param_s3_start, floating=floating)
        param_p5 = zfit.Parameter(f'param_p5_{r}', param_p5_start, floating=floating)

        extended = zfit.Parameter(f"extended_{r}", 10000)

        pdf_3D = p5_prime_pdf(param_fl=param_fl, param_s3=param_s3, param_p5=param_p5, obs=obs, extended=extended)
        pdf_3D.update_integration_options(max_draws=10000, tol=1e-4)  # Adjust parameters as needed

        # create NLL
        nll = zfit.loss.ExtendedUnbinnedNLL(model=pdf_3D, data=data_fit_sig)
        minimizer = zfit.minimize.Minuit()
        result = minimizer.minimize(nll)

        if floating == True:
            result.hesse(name='hesse')

        print(result.params)
        print("Fit result valid:")
        print(result.valid)

        return pdf_3D
    
    mc_signal_data_with_trim_fold = fold_data(mc_signal_data)

    mc_signal_data_with_trim_fold_weight=mc_signal_data_with_trim_fold["PIDCorrectionAndAcceptanceWeight"]

    mc_signal_data_with_trim_weight_fold=sum(mc_signal_data_with_trim_fold_weight)

    # Example usage with adjusted starting parameters
    param_fl_start = 0.5  # Adjust based on your data 0.5
    param_s3_start = 0.01  # Adjust based on your data -0.01
    param_p5_start = 0.02  # Adjust based on your data 0.02

    # Fit with phi_B greater than np.pi

    pdf_fit_signal_with_trim_gr = p5p_3D_fit(mc_signal_data_with_trim_fold["cosTheta_K"], mc_signal_data_with_trim_fold["cosTheta_L"], mc_signal_data_with_trim_fold["phi_B"],
                                   param_fl_start, param_s3_start, param_p5_start,weight=mc_signal_data_with_trim_fold_weight,
                                   floating=True)


    # Now, let's print the total signal events again
    print("Signal Events in MC Data =", mc_signal_data_with_trim_weight_fold)
    
    # Define ranges
    cosTheta_K_range_with_trim = np.linspace(-0.9, 1.0, 100)  # Range for cosTheta_K with trim
    cosTheta_K_range_without_trim = np.linspace(-1.0, 1.0, 100)  # Range for cosTheta_K without trim
    cosTheta_L_range = np.linspace(0, 1.0, 100)
    phi_B_range = np.linspace(0, np.pi, 100)
    
    # Create grids for with trim
    cosTheta_K_grid_with_trim, cosTheta_L_grid, phi_B_grid = np.meshgrid(cosTheta_K_range_with_trim, cosTheta_L_range, phi_B_range)
    points_with_trim = np.vstack([cosTheta_K_grid_with_trim.flatten(), cosTheta_L_grid.flatten(), phi_B_grid.flatten()]).T

    # Create grids for without trim
    cosTheta_K_grid_without_trim,cosTheta_L_grid_without_trim, phi_B_grid_without_trim= np.meshgrid(cosTheta_K_range_without_trim, cosTheta_L_range, phi_B_range)
    points_without_trim = np.vstack([cosTheta_K_grid_without_trim.flatten(), cosTheta_L_grid_without_trim.flatten(), phi_B_grid_without_trim.flatten()]).T

        
    # PDF values without trim
    pdf_values_gr_without_trim = pdf_fit_signal_with_trim_gr.pdf(points_without_trim).numpy().reshape(cosTheta_K_grid_without_trim.shape)
    integrated_pdf_total_without_trim = simps(simps(simps(pdf_values_gr_without_trim, phi_B_range, axis=2), cosTheta_L_range, axis=1), cosTheta_K_range_without_trim, axis=0)
    
    # PDF values with trim
    pdf_values_gr_with_trim = pdf_fit_signal_with_trim_gr.pdf(points_with_trim).numpy().reshape(cosTheta_K_grid_with_trim.shape)
    integrated_pdf_total_with_trim = simps(simps(simps(pdf_values_gr_with_trim, phi_B_range, axis=2), cosTheta_L_range, axis=1), cosTheta_K_range_with_trim, axis=0)

    # Calculate proportion of data lost
    proportion_data_lost = (integrated_pdf_total_without_trim - integrated_pdf_total_with_trim) / integrated_pdf_total_without_trim

    # Print the integrated PDF
    print(f"Integrated PDF with trim: {integrated_pdf_total_with_trim}")
    print(f"Integrated PDF without trim: {integrated_pdf_total_without_trim}")
    print(f"Proportion of data lost: {proportion_data_lost:.4f}")


In [10]:
def fit(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, mc_label_particle_type_dataset_name_weight, model_type,q2_reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight

    r = round(np.random.rand(), 4)
    ratios_label_dataset_model_type = None
        

    if model_type == 'cb':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", 5.279)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", 0.005)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", 1)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", 4)
        
        
        label_dataset_name_model_type = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
    
    elif model_type == 'cb_gauss':
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", -0.0)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", 0.09)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", 1)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", 2)

        label_dataset_name_cb = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", -0.0)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.096)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb, label_dataset_name_gauss],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.29, 5.28, 5.3)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-60)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 0.5)

        label_dataset_name_argus = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", 5.278)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.005)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus,label_dataset_name_gauss],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_poly = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.289)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-120)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 0.5)

        label_dataset_name_argus = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly, label_dataset_name_argus],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
        
    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_mu_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_mu_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_poly = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_mu_label_dataset_name_poly, coeff1_mu_label_dataset_name_poly])

        # Gaussian
        mu_mu_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.0)
        sigma_mu_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.08)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_mu_label_dataset_name_gauss, sigma=sigma_mu_label_dataset_name_gauss)
        
        # Fraction
        frac_mu_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly,label_dataset_name_gauss],
            fracs=[frac_mu_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.289)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-120)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 0.5)

        label_dataset_name_model_type = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_model_type = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])

    
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
        
    
    data_label_dataset_name = zfit.data.Data.from_pandas(
    mc_dataset_name_data_label,
    obs=obs_label,
    weights=mc_dataset_name_weight_label
    )
   
    nll_label_dataset_name = zfit.loss.UnbinnedNLL(model=label_dataset_name_model_type, data=data_label_dataset_name)
    minimizer_label_dataset_name = zfit.minimize.Minuit()
    result_label_dataset_name = minimizer_label_dataset_name.minimize(nll_label_dataset_name)
    
    # Extract and save the parameter values
    parameters_values = {
        "_".join(param.name.split('_')[:-1]): param.numpy() for param in label_dataset_name_model_type.get_params()
    }
    ratios_label_dataset_model_type = pd.DataFrame(parameters_values, index=[0])
    ratios_label_dataset_model_type.to_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2_reg}.csv", index=False)
    
    return label_dataset_name_model_type,result_label_dataset_name


In [11]:
def plot_with_fit(ax, bins, data, data_weight, fitted_model, data_range, label_data, label_fit, xlabel, ylabel, title):
    # Calculate histogram
    n, bins, _ = ax.hist(data, bins=bins, weights=data_weight, density=False, alpha=0.5, label=label_data)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Calculate errors
    bin_weights = []
    for i in range(len(bins) - 1):
        bin_indices = np.where((data >= bins[i]) & (data < bins[i+1]))[0]
        bin_weights.append(data_weight.iloc[bin_indices].tolist())
    errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]

    # De-normalization
    pdf_values = fitted_model.pdf(data_range).numpy() * sum(data_weight) * (bins[1] - bins[0])

    # Plot histogram with error bars and fitted model
    ax.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label=f'{label_data} with Error Bars')
    ax.plot(data_range, pdf_values, 'r', label=label_fit)

    # Add labels and title
    ax.legend(fontsize=15)
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    ax.set_title(title, fontsize=15)

    return pdf_values

In [12]:
def fit_fixed(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, mc_label_particle_type_dataset_name_weight, model_type,q2_reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight

    r = round(np.random.rand(), 4)
    
    parameter_df = pd.read_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2_reg}.csv")
    
    if model_type == 'cb':
        
        # CrystalBall

        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
      
        label_dataset_name_model_type_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
    elif model_type == 'cb_gauss':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)

        label_dataset_name_cb_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb_fixed, label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        
        
        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
  
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed, label_dataset_name_argus_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label
        )
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.28819,floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
    
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0],floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    
    return label_dataset_name_model_type_fixed

In [13]:
def fit_fixed_without_weight(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, model_type,q2_reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        

    r = round(np.random.rand(), 4)
    
    parameter_df = pd.read_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2_reg}.csv")
    
    if model_type == 'cb':
        
        # CrystalBall

        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
      
        label_dataset_name_model_type_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
    elif model_type == 'cb_gauss':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)

        label_dataset_name_cb_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb_fixed, label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        
        
        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
  
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",parameter_df[f"frac_{particle_type}_{label}_{dataset_name}_{model_type}"].values[0],floating=False)        

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed, label_dataset_name_argus_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label
        )
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.28819,floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
    
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0],floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    
    return label_dataset_name_model_type_fixed

In [14]:
def fit_fixed_unfixed_data(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, mc_label_particle_type_dataset_name_weight, model_type,q2_reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight

    r = round(np.random.rand(), 4)
    
    parameter_df = pd.read_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2_reg}.csv")
    
    if model_type == 'cb':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", 5.279)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
      
        label_dataset_name_model_type_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
    elif model_type == 'cb_gauss':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"], floating=False)

        label_dataset_name_cb_fixed = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",0.91)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb_fixed, label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}",0.002)
        
        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)

    
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"], floating=False)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.894)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus_fixed,label_dataset_name_gauss_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
  
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed, label_dataset_name_argus_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label
        )
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.28819 )
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
    
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=True)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
        
    data_label_dataset_name = zfit.data.Data.from_pandas(mc_label_particle_type_dataset_name_data, obs=obs_label, weights=mc_label_particle_type_dataset_name_weight)
    nll_label_dataset_name = zfit.loss.UnbinnedNLL(model=label_dataset_name_model_type_fixed, data=data_label_dataset_name)
    minimizer_label_dataset_name = zfit.minimize.Minuit()
    result_label_dataset_name = minimizer_label_dataset_name.minimize(nll_label_dataset_name)
    
    return label_dataset_name_model_type_fixed

In [15]:
def fit_extended_model(Mbc_signal_pdf, deltaE_signal_pdf,Mbc_bkg_pdf, deltaE_bkg_pdf, 
                       data, weights,  yield_signal, yield_background):
    
    obs_Mbc = zfit.Space('Mbc', limits=(5.20, 5.29))
    obs_deltaE = zfit.Space('deltaE', limits=(-0.15, 0.15))
    
    combined_obs = obs_Mbc * obs_deltaE
    
    # Random number for parameter names
    r = np.round(np.random.rand(), decimals=4)
    
    # Signal model
    Mbc_signal_pdf = Mbc_signal_pdf
    deltaE_signal_pdf = deltaE_signal_pdf
    signal_product_pdf = zfit.pdf.ProductPDF([Mbc_signal_pdf, deltaE_signal_pdf])
    signal_yield = zfit.Parameter(f"signal_yield_{r}", yield_signal)
    signal_extended = signal_product_pdf.create_extended(signal_yield)

    # Background model 
    Mbc_background_pdf = Mbc_bkg_pdf
    deltaE_background_pdf = deltaE_bkg_pdf

    background_product_pdf = zfit.pdf.ProductPDF([Mbc_background_pdf, deltaE_background_pdf])
    background_yield = zfit.Parameter(f"background_yield_{r}", yield_background)
    background_extended = background_product_pdf.create_extended(background_yield)

    # Total model sum
    total_model = zfit.pdf.SumPDF([signal_extended, background_extended])
    
    # Create a joint data for Mbc and deltaE
    combined_data = zfit.data.Data.from_pandas(data, obs=combined_obs, weights=weights)

    # Create an extended NLL for the model and data
    nll = zfit.loss.ExtendedUnbinnedNLL(model=total_model, data=combined_data)

    # Minimize the NLL
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    
    return Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result



In [16]:
def fit_extended_model_scf_without_weight(Mbc_only_signal_pdf, Mbc_scf_pdf, deltaE_only_signal_pdf, deltaE_scf_pdf,
                           Mbc_bkg_pdf, deltaE_bkg_pdf, data, frc_signal,frc_background,yield_signal, yield_background):
    
    obs_Mbc = zfit.Space('Mbc', limits=(5.20, 5.29))
    obs_deltaE = zfit.Space('deltaE', limits=(-0.15, 0.15))
    
    combined_obs = obs_Mbc * obs_deltaE
    
    # Random number for parameter names
    r = np.round(np.random.rand(), decimals=4)
    
    # Signal model
    Mbc_signal_pdf = Mbc_only_signal_pdf
    deltaE_signal_pdf = deltaE_only_signal_pdf
    signal_product_pdf = zfit.pdf.ProductPDF([Mbc_signal_pdf, deltaE_signal_pdf])
    signal_yield = zfit.Parameter(f"signal_yield_{r}", yield_signal)
    signal_extended = signal_product_pdf.create_extended(signal_yield)

    # Background model
    frac_Mbc_bkg1 = zfit.Parameter(f'frac_Mbc_bkg1_{r}', frc_background)
    frac_deltaE_bkg1 = zfit.Parameter(f'frac_deltaE_bkg1_{r}', frc_background)
        
    Mbc_background_pdf = zfit.pdf.SumPDF([Mbc_scf_pdf, Mbc_bkg_pdf], fracs=[frac_Mbc_bkg1])
    deltaE_background_pdf = zfit.pdf.SumPDF([deltaE_scf_pdf, deltaE_bkg_pdf], fracs=[frac_deltaE_bkg1])

    background_product_pdf = zfit.pdf.ProductPDF([Mbc_background_pdf, deltaE_background_pdf])
    background_yield = zfit.Parameter(f"background_yield_{r}", yield_background)
    background_extended = background_product_pdf.create_extended(background_yield)

    # Total model sum
    total_model = zfit.pdf.SumPDF([signal_extended, background_extended])
    
    # Create a joint data for Mbc and deltaE
    combined_data = zfit.data.Data.from_pandas(data, obs=combined_obs)

    # Create an extended NLL for the model and data
    nll = zfit.loss.ExtendedUnbinnedNLL(model=total_model, data=combined_data)

    # Minimize the NLL
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    
    return Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result


In [17]:
def fit_extended_model_scf(Mbc_only_signal_pdf, Mbc_scf_pdf, deltaE_only_signal_pdf, deltaE_scf_pdf,
                           Mbc_bkg_pdf, deltaE_bkg_pdf, data, weights, frc_signal,frc_background,yield_signal, yield_background):
    
    obs_Mbc = zfit.Space('Mbc', limits=(5.20, 5.29))
    obs_deltaE = zfit.Space('deltaE', limits=(-0.15, 0.15))
    
    combined_obs = obs_Mbc * obs_deltaE
    
    # Random number for parameter names
    r = np.round(np.random.rand(), decimals=4)
    
    # Signal model
    Mbc_signal_pdf = Mbc_only_signal_pdf
    deltaE_signal_pdf = deltaE_only_signal_pdf
    signal_product_pdf = zfit.pdf.ProductPDF([Mbc_signal_pdf, deltaE_signal_pdf])
    signal_yield = zfit.Parameter(f"signal_yield_{r}", yield_signal)
    signal_extended = signal_product_pdf.create_extended(signal_yield)

    # Background model
    frac_Mbc_bkg1 = zfit.Parameter(f'frac_Mbc_bkg1_{r}', frc_background)
    frac_deltaE_bkg1 = zfit.Parameter(f'frac_deltaE_bkg1_{r}', frc_background)
        
    Mbc_background_pdf = zfit.pdf.SumPDF([Mbc_scf_pdf, Mbc_bkg_pdf], fracs=[frac_Mbc_bkg1])
    deltaE_background_pdf = zfit.pdf.SumPDF([deltaE_scf_pdf, deltaE_bkg_pdf], fracs=[frac_deltaE_bkg1])

    background_product_pdf = zfit.pdf.ProductPDF([Mbc_background_pdf, deltaE_background_pdf])
    background_yield = zfit.Parameter(f"background_yield_{r}", yield_background)
    background_extended = background_product_pdf.create_extended(background_yield)

    # Total model sum
    total_model = zfit.pdf.SumPDF([signal_extended, background_extended])
    
    # Create a joint data for Mbc and deltaE
    combined_data = zfit.data.Data.from_pandas(data, obs=combined_obs, weights=weights)

    # Create an extended NLL for the model and data
    nll = zfit.loss.ExtendedUnbinnedNLL(model=total_model, data=combined_data)

    # Minimize the NLL
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    
    return Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result


In [18]:
def signal_yield_and_mc_results(result, signal_extended, background_extended, mc_signal_weight,mc_scf_weight ,mc_bkg_weight):
    # Extract and round yield values from fit results
    signal_yield_value = result.params[signal_extended.get_yield()]['value']
    background_yield_value = result.params[background_extended.get_yield()]['value']
    
    signal_event_fit = round(signal_yield_value, 3)
    bkg_event_fit = round(background_yield_value, 3)
    
    # Parameters from data (example values, adjust as needed)
    MC_signal = round(sum(mc_ele_only_signal_weight), 3)
    MC_bkg = round(sum(pd.concat([mc_ele_scf_weight,mc_ele_bkg_weight])), 3)

    # Print fit results
    print("Results from Fit:")
    print("Signal Events =", signal_event_fit)
    print("Background Events =", bkg_event_fit)
    print()
    
    # Print MC data results
    print("Results from MC Data:")
    print("Signal Events in MC Data =", MC_signal)
    print("Background Events in MC Data =", MC_bkg)
    print()
    
    # Calculate and print differences
    errors_sig_events = MC_signal - signal_event_fit
    errors_bkg_events = MC_bkg - bkg_event_fit
    print("Signal Events difference =", round(errors_sig_events, 3))
    print("Background Events difference =", round(errors_bkg_events, 3))



In [19]:
def process_fit_and_mc_results(result, signal_extended, background_extended, mc_signal_weight, mc_bkg_weight, 
                               q2reg, ll, correction_factor, mp, dp, apply_correction=True):
    # Extract and round yield values from fit results
    signal_yield_value = result.params[signal_extended.get_yield()]['value']
    background_yield_value = result.params[background_extended.get_yield()]['value']
    
    signal_event_fit = round(signal_yield_value, 3)
    bkg_event_fit = round(background_yield_value, 3)
    
    # Parameters from data (example values, adjust as needed)
    MC_signal = round(sum(mc_signal_weight), 3)
    MC_bkg = round(sum(mc_bkg_weight), 3)

    # Print fit results
    print("Results from Fit:")
    print("Signal Events =", signal_event_fit)
    print("Background Events =", bkg_event_fit)
    print()
    
    # Print MC data results
    print("Results from MC Data:")
    print("Signal Events in MC Data =", MC_signal)
    print("Background Events in MC Data =", MC_bkg)
    print()
    
    # Calculate and print differences
    errors_sig_events = MC_signal - signal_event_fit
    errors_bkg_events = MC_bkg - bkg_event_fit
    print("Signal Events difference =", round(errors_sig_events, 3))
    print("Background Events difference =", round(errors_bkg_events, 3))
    
    # Calculate signal yield uncertainty
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2
    signal_yield_with_uncertainty = ufloat(signal_yield_value, signal_yield_error)
    
    # Constants for branching ratio calculation
    cross_section_nb = 0.5346e-9
    luminosity_nb = 368.4e+15
    N_bb_mesons = cross_section_nb * (4 * luminosity_nb)

    # Define PDG values for branching ratios
    BR_PDG_Bp_JPsi = ufloat(1.43e-3, 0.08e-3)
    BR_PDG_Bp_Psi = ufloat(6.7e-4, 1.4e-4)
    BR_PDG_B0_JPsi = ufloat(1.27e-3, 0.05e-3)
    BR_PDG_B0_Psi = ufloat(5.9e-3, 0.4e-3)

    # Determine B_K_to_K_pi based on mp (Bp or B0)
    if mp == 'Bp':
        B_K_to_K_pi = (ufloat(99.754, 0.021) / 100) * 1/3
    elif mp == 'B0':
        B_K_to_K_pi = (ufloat(99.754, 0.021) / 100) * 2/3
    else:
        print("Invalid value of mp. Please provide 'Bp' or 'B0'.")
        return
    
    # Determine B_JPsi or B_Psi based on q2reg and ll
    if q2reg == 11:
        if ll == 'ee':
            B_JPsi = ufloat(5.971, 0.032) / 100
        elif ll == 'mumu':
            B_JPsi = ufloat(5.961, 0.033) / 100
        else:
            print("Invalid value of ll. Please provide 'ee' or 'mumu'.")
            return
        
        # BR for fit data
        if apply_correction:
            adjusted_signal_yield = signal_yield_with_uncertainty * (1 + correction_factor)
        else:
            adjusted_signal_yield = signal_yield_with_uncertainty
            
        BR_MC = adjusted_signal_yield / ((2 * N_bb_mesons) * B_JPsi * B_K_to_K_pi)
        print("B_to_K*_JPsi_Branching_ratio(Fit Data):", BR_MC)

        # BR for MC data
        if apply_correction:
            adjusted_MC_signal = MC_signal * (1 + correction_factor)
        else:
            adjusted_MC_signal = MC_signal
            
        BR_MC_from_MC = adjusted_MC_signal / ((2 * N_bb_mesons) * B_JPsi * B_K_to_K_pi)
        print("B_to_K*_JPsi_Branching_ratio(MC Data):", BR_MC_from_MC)
        print(f"B_to_K*_JPsi_Branching_ratio_from_PDG: {eval(f'BR_PDG_{mp}_{dp}')}")

    elif q2reg == 12:
        if ll == 'ee':
            B_Psi = ufloat(7.94e-3, 0.22e-3)
        elif ll == 'mumu':
            B_Psi = ufloat(8.0e-3, 0.6e-3)
        else:
            print("Invalid value of ll. Please provide 'ee' or 'mumu'.")
            return
        
        # BR for fit data
        if apply_correction:
            adjusted_signal_yield = signal_yield_with_uncertainty * (1 + correction_factor)
        else:
            adjusted_signal_yield = signal_yield_with_uncertainty
            
        BR_MC = adjusted_signal_yield / ((2 * N_bb_mesons) * B_Psi * B_K_to_K_pi)
        print("B_to_K*_Psi_Branching_ratio(Fit Data):", BR_MC)
    
        # BR for MC data
        if apply_correction:
            adjusted_MC_signal = MC_signal * (1 + correction_factor)
        else:
            adjusted_MC_signal = MC_signal
            
        BR_MC_from_MC = adjusted_MC_signal / ((2 * N_bb_mesons) * B_Psi * B_K_to_K_pi)
        print("B_to_K*_Psi_Branching_ratio(MC Data):", BR_MC_from_MC)
        print(f"B_to_K*_Psi_Branching_ratio_from_PDG: {eval(f'BR_PDG_{mp}_{dp}')}")
    else:
        print("Invalid value of q2reg. Please provide a valid condition (11 or 12).")
        

In [20]:
def plot_variable(ax, ax_pull, variable, data_signal, data_scf, data_bkg, weight_signal, weight_scf, weight_bkg, 
                  signal_pdf, background_pdf, label_signal, label_background,label_total, xaxis):    

    signal_df = data_signal[variable]
    bkg_df = pd.concat([data_scf[variable], data_bkg[variable]])
    
    df_total = pd.concat([signal_df, bkg_df])
    
    signal_weight = weight_signal
    bkg_weight = pd.concat([weight_scf, weight_bkg])
    
    total_weight = pd.concat([signal_weight, bkg_weight])
 
    # Define bin edges and x_fit
    if variable == "Mbc":
        bin_edges = np.linspace(5.20, 5.289, 100)
        x_fit = np.linspace(5.20, 5.289, 100)
        ax.set_xlim(5.20, 5.29)
        ax_pull.set_xlim(5.20, 5.29)
        
    elif variable == "deltaE":
        bin_edges = np.linspace(-0.15, 0.15, 100)
        x_fit = np.linspace(-0.15, 0.15, 100)
        ax.set_xlim(-0.15, 0.15)
        ax_pull.set_xlim(-0.15, 0.15)

    # Plot the background component
    y_background_fit = background_pdf.pdf(x_fit) * sum(bkg_weight) * np.diff(bin_edges)[0]
    ax.plot(x_fit, y_background_fit, label=label_background, linestyle="--", color="blue", linewidth=2)

    # Plot the signal component
    y_signal_fit = signal_pdf.pdf(x_fit) * sum(signal_weight) * np.diff(bin_edges)[0]
    ax.plot(x_fit, y_signal_fit, label=label_signal, linestyle="-", color="red", linewidth=2)

    # Plot the total fit
    n, bins, _ = ax.hist(df_total, bins=bin_edges, weights=total_weight, density=False, color='white', alpha=0.3, edgecolor='black')
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Calculate errors
    bin_weights = []
    for i in range(len(bin_edges) - 1):
        bin_indices = np.where((df_total >= bins[i]) & (df_total < bins[i+1]))[0]
        bin_weights.append(total_weight.iloc[bin_indices].tolist())
    errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]

    # Plot histogram with error bars and fitted model
    ax.errorbar(bin_centers, n, yerr=errors, fmt='o', markersize=5, color='black', label=label_total, capsize=3)

    # Plot the total fit with error bars
    y_total_fit = y_signal_fit + y_background_fit
    
    # Adjust y_total_fit to match the number of bins
    y_total_fit_binned = np.interp(bin_centers, x_fit, y_total_fit)
    ax.plot(x_fit, y_total_fit, label="Total Fit", color="black", linewidth=2)

    # Calculate pull and plot
    pulls = (n - y_total_fit_binned) / errors
    ax_pull.bar(bin_centers, pulls, width=np.diff(bin_edges), align='center', alpha=0.5, color='gray')
    ax_pull.axhline(0, color='black', linewidth=1)

    # Add labels, legend, and grid with increased font sizes
    ax_pull.set_xlabel(xaxis, fontsize=20)
    ax_pull.set_ylabel('Pull', fontsize=20)
    ax_pull.tick_params(axis='both', which='major', labelsize=14)
    ax_pull.grid(True, linestyle='--', alpha=0.7)
    
    ax.set_ylabel('Counts', fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.legend(fontsize=15)
    ax.grid(True, linestyle='--', alpha=0.7)

    # Optional: Adjust figure size if needed
    fig = ax.get_figure()
    fig.set_size_inches(17,9)  # Adjust the size as needed
